# Library

In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# Connection

In [2]:
conn = psycopg2.connect(
    host="127.0.0.1",
    port="5432",
    database="olist",
    user="postgres",
    password="0000"
)

In [3]:
engine = create_engine('postgresql+psycopg2://', creator=lambda: conn)

# Import Data

In [4]:
datageoloc = pd.read_csv('data/olist_geolocation_dataset.csv')

In [5]:
datageoloc.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


# Write in database

## State

In [11]:
table_name = 'olist_state'

In [18]:
state = pd.DataFrame(datageoloc['geolocation_state'].unique(), columns=['state_code'])

In [19]:
state

,state_code
0,SP
1,RN
2,AC
3,RJ
4,ES
5,MG
6,BA
7,SE
8,PE
9,AL


In [20]:
state.to_sql(table_name, engine, if_exists='append', index=False)

27

## zip_code

In [23]:
table_name = 'olist_zipcode'

In [25]:
zipcode = pd.DataFrame('zip_code_prefix': datageoloc['geolocation_zip_code_prefix'].unique())

In [26]:
zipcode.head()

,zip_code_prefix
0,1037
1,1046
2,1041
3,1035
4,1012


In [29]:
zipcode.shape

(19015, 1)

In [30]:
zipcode.to_sql(table_name, engine, if_exists='append', index=False)

15

## City

In [31]:
table_name = 'olist_city'

In [32]:
select_col = ['geolocation_city', 'geolocation_state']

In [34]:
city = datageoloc[select_col].drop_duplicates()

In [35]:
city.head()

,geolocation_city,geolocation_state
0,sao paulo,SP
5,são paulo,SP
10166,sao bernardo do campo,SP
22261,são paulo,RN
50999,jundiaí,SP


In [37]:
city.shape

(8463, 2)

### Renommer les colonnes

In [41]:
cols_name = ['city_name', 'state_code']
city.columns = cols_name

### Ajout dans la bdd

In [42]:
city.to_sql(table_name, engine, if_exists='append', index=False)

463

## cityzipcode (Asso 17)

In [43]:
table_name = 'olist_cityzipcode'

In [44]:
select_col = ['geolocation_city', 'geolocation_state', 'geolocation_zip_code_prefix']

In [45]:
cityzipcode = datageoloc[select_col].drop_duplicates()

In [47]:
cityzipcode.head()

,geolocation_city,geolocation_state,geolocation_zip_code_prefix
0,sao paulo,SP,1037
1,sao paulo,SP,1046
3,sao paulo,SP,1041
4,sao paulo,SP,1035
5,são paulo,SP,1012


### Get city_id

In [80]:
cur = conn.cursor()

In [92]:
query = "SELECT city_id FROM olist_city WHERE city_name = %s AND state_code = %s"

In [108]:
def getCityId(row):
    # Exécution de la requête SQL pour récupérer la valeur correspondant aux valeurs de colonne1 et colonne2 de la ligne
    cur.execute(query, (row["city_name"], row["state_code"]))
    value = cur.fetchone()[0]
    # Retourne la valeur pour ajouter une nouvelle colonne au DataFrame
    return value

cityzipcode["city_id"] = cityzipcode.apply(getCityId, axis=1)

In [110]:
cityzipcode

,geolocation_city,geolocation_state,geolocation_zip_code_prefix,city_id
0,sao paulo,SP,1037,1
1,sao paulo,SP,1046,1
3,sao paulo,SP,1041,1
4,sao paulo,SP,1035,1
5,são paulo,SP,1012,2
...,...,...,...,...
999806,ibiaçá,RS,99940,8459
999846,santa cecilia do sul,RS,99952,8460
999867,ciríaco,RS,99970,8461
999891,estação,RS,99930,8462


### Renommer les colonnes

In [113]:
nouveaux_noms = {
    'geolocation_zip_code_prefix': 'zip_code_prefix'
}
cityzipcode.rename(columns=nouveaux_noms, inplace=True)

In [115]:
czc = cityzipcode[["city_id", "zip_code_prefix"]]

In [117]:
czc.head()

,city_id,zip_code_prefix
0,1,1037
1,1,1046
3,1,1041
4,1,1035
5,2,1012


### Add in data base

In [118]:
czc.to_sql(table_name, engine, if_exists='append', index=False)

912

## geolocation

In [23]:
table_name = 'olist_geolocation'

### Get city_id from db

In [7]:
cities_db = pd.read_sql("SELECT * FROM olist_city", engine)

In [8]:
cities_db.head()

,city_id,city_name,state_code
0,1,sao paulo,SP
1,2,são paulo,SP
2,3,sao bernardo do campo,SP
3,4,são paulo,RN
4,5,jundiaí,SP


### Get Data

In [9]:
dfgeoloc = datageoloc.iloc[:, 1:]

In [10]:
cols_name = ['geolocation_lat', "geolocation_lng", "city_name", "state_code"]
dfgeoloc.columns = cols_name

In [11]:
dfgeoloc.head()

,geolocation_lat,geolocation_lng,city_name,state_code
0,-23.545621,-46.639292,sao paulo,SP
1,-23.546081,-46.644820,sao paulo,SP
2,-23.546129,-46.642951,sao paulo,SP
3,-23.544392,-46.639499,sao paulo,SP
4,-23.541578,-46.641607,sao paulo,SP


In [12]:
dfgeoloc.shape

(1000163, 4)

In [13]:
cur = conn.cursor()

In [17]:
query = "SELECT city_id FROM olist_city WHERE city_name = %s AND state_code = %s"

In [18]:
def getCityId(row):
    # Exécution de la requête SQL pour récupérer la valeur correspondant aux valeurs de colonne1 et colonne2 de la ligne
    cur.execute(query, (row["city_name"], row["state_code"]))
    value = cur.fetchone()[0]
    # Retourne la valeur pour ajouter une nouvelle colonne au DataFrame
    return value

dfgeoloc["city_id"] = dfgeoloc.apply(getCityId, axis=1)

In [19]:
dfgeoloc.head()

,geolocation_lat,geolocation_lng,city_name,state_code,city_id
0,-23.545621,-46.639292,sao paulo,SP,1
1,-23.546081,-46.644820,sao paulo,SP,1
2,-23.546129,-46.642951,sao paulo,SP,1
3,-23.544392,-46.639499,sao paulo,SP,1
4,-23.541578,-46.641607,sao paulo,SP,1


In [22]:
geoloc = dfgeoloc[["geolocation_lng", "geolocation_lat", "city_id"]]

In [24]:
geoloc.to_sql(table_name, engine, if_exists='append', index=False)

163

# Close the cursor and connection objects

In [25]:
conn.commit()
cur.close()
conn.close()